# 필요한 라이브러리, 데이터 로드

In [1]:
import pandas as pd
import re
pd.options.plotting.backend = "plotly"
import cufflinks as cf
cf.go_offline(connected=True)
from more_itertools import locate,replace

In [2]:
izone_df = pd.read_csv('./data/izone_youtube_preprocessed.csv',encoding='utf-8-sig')
izone_df

,제목,업로더,조회수,게시일
0,엠넷 재팬 MC 本田仁美 혼다히토미 아이즈원 언급 한글자막,곽군테레비 郭君テレビ,11330,2021-12-28
1,아이즈원,강민혁 X 차누,3727913,2021-03-25
2,"🎵 아이즈원 (IZONE) 노래모음 추천곡 (라비앙로즈, FIESTA, 비올레타, ...",박승오리 Box,158297,2020-11-05
3,아이즈원(IZ*ONE) - Panorama(파노라마) 교차편집(stage mix),강민혁 X 차누,5157126,2021-03-24
4,IZ*ONE (아이즈원) 'Panorama' MV,Stone Music Entertainment,69191968,2020-12-07
...,...,...,...,...
523,"[아이즈원]은비 당황, LA에도 내 취향은 없어요",쌈조TV3,2553,2021-12-31
524,"[아이즈원]초딩실직, 너무 귀여운 예나",쌈조TV3,2035,2021-12-31
525,2021년 베스트 걸그룹 1위!! 아이즈원 삼성동 광고,곽군테레비 郭君テレビ,1916,2021-12-29
526,[아이즈원]드디어 뭉쳤다 3사 MC즈!,쌈조TV3,22516,2021-12-31


# 조회수 상위 10위 유튜버

In [3]:
izone_youtuber = izone_df['조회수'].groupby(izone_df['업로더']).sum().to_frame().sort_values(by='조회수',ascending=False).head(11)

In [4]:
izone_youtuber

,조회수
업로더,
Stone Music Entertainment,504442750
official IZ*ONE,224430839
M2,200632526
강민혁 X 차누,48620600
Mnet K-POP,46965047
KBS Kpop,32551637
STUDIO CHOOM [스튜디오 춤],26381460
Jaeguchi,25208289
그린채널,20879102


In [5]:
# 솔연님의 눈부신 성과
izone_youtuber.iloc[1:].plot.bar()

In [6]:
# 교차편집 맛집
izone_df[izone_df['업로더']=='SoulYeon솔연']

,제목,업로더,조회수,게시일
252,아이즈원 (IZ*ONE) Panorama (파노라마) 교차편집 (Stage Mix),SoulYeon솔연,3659885,2021-01-02


# 조회수 분석

## MV가 높은 조회수

In [7]:
izone_view_df = izone_df['조회수'].groupby(izone_df['게시일']).sum()
izone_view_df.plot()

## izone 공식 유튜브 제외

In [8]:
non_official_df = izone_df[izone_df['업로더']!='izone']
non_official_df = non_official_df[non_official_df['업로더']!='1theK (원더케이)']

In [9]:
non_official_view_df= non_official_df['조회수'].groupby(izone_df['게시일']).sum()
non_official_view_df.plot()

In [10]:
#200만 뷰를 넘은 영상은?
non_official_df[non_official_df['조회수']>2000000]

,제목,업로더,조회수,게시일
1,아이즈원,강민혁 X 차누,3727913,2021-03-25
3,아이즈원(IZ*ONE) - Panorama(파노라마) 교차편집(stage mix),강민혁 X 차누,5157126,2021-03-24
4,IZ*ONE (아이즈원) 'Panorama' MV,Stone Music Entertainment,69191968,2020-12-07
5,[4회] 아홉 개의 보석! '크라운' ♬FIESTA_IZ*ONE(아이즈원) @CON...,Mnet K-POP,9694389,2021-08-27
6,[ENG/JPN] 🧐마피아게임 하다가 단체로 드러누운(?) 아이즈원 | IZ*ONE...,딩고 뮤직 / dingo music,5807974,2020-12-13
...,...,...,...,...
511,[입덕직캠] 아이즈원 미야와키 사쿠라 직캠 4K 'Panorama' (IZ*ONE ...,M2,2186092,2020-12-20
513,[MPD직캠] 아이즈원 김채원 직캠 4K ‘Panorama’ (IZ*ONE Kim ...,M2,2489050,2020-12-08
515,[Artist Of The Month] '16 Shots' covered by IZ...,STUDIO CHOOM [스튜디오 춤],11180199,2020-10-05
520,[입덕직캠] 아이즈원 안유진 직캠 4K ‘Sequence’ (IZ*ONE Ahn Y...,M2,3601004,2020-12-11


# 가장 열심히 영상을 올려준 상위 10인의 유튜버는?

In [11]:
uploader_count = izone_df['업로더'].groupby(izone_df['업로더']).count().to_frame()

In [12]:
uploader_count.columns = ['카운트']
uploader_count.sort_values(by='카운트',ascending=False).head(20)

,카운트
업로더,
M2,72
쌈조TV3,56
official IZ*ONE,25
Mnet K-POP,21
Stone Music Entertainment,18
KBS Kpop,17
스브스케이팝 / SBS KPOP,17
강민혁 X 차누,14
ALL THE K-POP,10
